## Data Processing

In [1]:
import numpy as np
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import os

def set_seed(seed=21019):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

2024-03-28 07:23:01.451901: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 07:23:01.452056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 07:23:01.619418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.15.0


In [2]:
import pandas as pd

forecast_part1 = '/kaggle/input/stackedlstm-part-1/forecast_part1.csv'
forecast_df = pd.read_csv(forecast_part1)
forecast_df.head(35)

,Country Name,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],...,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024],2025 [YR2025]
0,Afghanistan,0.055167,0.055293,0.066810,0.073005,0.054867,0.080361,0.089757,0.109940,0.165573,...,0.333432,0.336906,0.337917,0.341122,0.342651,0.335550,0.341278,0.345148,0.342827,0.347666
1,Albania,1.031568,1.056868,1.233002,1.361159,1.427944,1.360092,1.363989,1.433797,1.428250,...,1.882345,1.912801,1.978082,2.082341,1.975886,2.060485,2.125400,2.339987,2.390953,2.573139
2,Algeria,2.601065,2.520610,2.605695,2.751099,2.752759,2.857927,2.985149,3.023503,3.116771,...,3.609105,4.344381,4.202256,4.837109,4.968634,5.259593,5.870263,5.128977,5.999907,6.506385
3,American Samoa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Andorra,7.927016,7.725627,7.498765,7.238069,7.286756,7.206274,6.805400,6.890390,7.081822,...,6.442749,6.553359,6.345446,6.329783,6.362727,5.633969,5.474488,5.635873,5.389369,4.770454
5,Angola,0.988416,0.941818,0.895717,0.924988,0.929811,0.812875,0.821264,0.811414,0.888610,...,1.077208,1.160938,1.168170,1.175627,1.210046,1.209676,1.281442,1.193666,1.286670,1.315567
6,Antigua and Barbuda,4.163613,4.223578,4.340955,4.619917,4.654109,4.716473,4.919958,5.111198,5.142281,...,6.045279,5.606689,6.255174,6.330445,6.598771,7.001918,6.582711,7.083184,7.537114,7.561091
7,Argentina,3.567918,3.341877,3.100489,3.334891,3.656085,3.736717,3.924262,4.083151,4.152375,...,4.635539,4.939681,4.991462,5.280142,5.429144,5.724504,5.925918,6.237144,6.548238,7.259562
8,Armenia,1.123867,1.149648,1.004400,1.133361,1.227695,1.463912,1.483304,1.732263,1.905698,...,2.024358,2.030762,1.976409,2.209575,2.277747,2.250194,2.268548,2.299278,2.477514,2.599467
9,Aruba,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [3]:
years = [f"{year} [YR{year}]" for year in range(2026, 2031)]

# Add columns '2016 [YR2016]' to '2030 [YR2030]' filled with 0 to co2_df
forecast_df = forecast_df.assign(**{year: 0 for year in years})
forecast_df.reset_index(drop=True, inplace=True)
forecast_df

,Country Name,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],...,2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024],2025 [YR2025],2026 [YR2026],2027 [YR2027],2028 [YR2028],2029 [YR2029],2030 [YR2030]
0,Afghanistan,0.055167,0.055293,0.066810,0.073005,0.054867,0.080361,0.089757,0.109940,0.165573,...,0.335550,0.341278,0.345148,0.342827,0.347666,0,0,0,0,0
1,Albania,1.031568,1.056868,1.233002,1.361159,1.427944,1.360092,1.363989,1.433797,1.428250,...,2.060485,2.125400,2.339987,2.390953,2.573139,0,0,0,0,0
2,Algeria,2.601065,2.520610,2.605695,2.751099,2.752759,2.857927,2.985149,3.023503,3.116771,...,5.259593,5.870263,5.128977,5.999907,6.506385,0,0,0,0,0
3,American Samoa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
4,Andorra,7.927016,7.725627,7.498765,7.238069,7.286756,7.206274,6.805400,6.890390,7.081822,...,5.633969,5.474488,5.635873,5.389369,4.770454,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sub-Saharan Africa,0.742866,0.787828,0.770312,0.793664,0.816417,0.799087,0.776663,0.776076,0.804147,...,0.777017,0.775042,0.773692,0.772140,0.772609,0,0,0,0,0
262,Sub-Saharan Africa (excluding high income),0.742518,0.787451,0.769905,0.793283,0.815994,0.798675,0.776253,0.775658,0.803729,...,0.776986,0.775146,0.772611,0.773846,0.772464,0,0,0,0,0
263,Sub-Saharan Africa (IDA & IBRD countries),0.742866,0.787828,0.770312,0.793664,0.816417,0.799087,0.776663,0.776076,0.804147,...,0.776683,0.775928,0.772067,0.773665,0.772653,0,0,0,0,0
264,Upper middle income,3.167141,3.243556,3.347642,3.646680,3.955406,4.246087,4.524940,4.770830,4.867978,...,6.375642,6.537148,6.622823,6.661193,6.817531,0,0,0,0,0


In [4]:
# List columns of forecast_df
forecast_columns = forecast_df.columns[27:]
print(forecast_columns)

Index(['2026 [YR2026]', '2027 [YR2027]', '2028 [YR2028]', '2029 [YR2029]',
       '2030 [YR2030]'],
      dtype='object')


# Create Model

In [5]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM, Dense

# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def lstm_univariate_prediction(raw_seq, n_steps, n_features, x_input):
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    # define model
    model = Sequential()
    model.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=220, verbose=0)
    # demonstrate prediction
    x_input = array(x_input)
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    del model
    return yhat

# Example usage:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
n_steps = 5
n_features = 1
x_input = [60, 70, 80, 90, 100]

# Perform multiple predictions and average the results
num_predictions = 5
predictions = np.zeros(num_predictions)

for i in range(num_predictions):
    predictions[i] = lstm_univariate_prediction(raw_seq, n_steps, n_features, x_input)

avg_prediction = np.mean(predictions)
print("Average Prediction:", avg_prediction)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/tmp/ipykernel_18/249675449.py:51: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[i] = lstm_univariate_prediction(raw_seq, n_steps, n_features, x_input)


Average Prediction: 110.10586853027344


# Forecasting

In [6]:
from tqdm.notebook import tqdm_notebook

# Define n_steps and n_features
n_steps = 8
n_features = 1

# Iterate over each column except 'Country Name'
for column in tqdm_notebook(forecast_columns, desc='Columns'):
    # Iterate over each row with tqdm_notebook
    for row in tqdm_notebook(range(len(forecast_df)), desc='Rows', leave=False):
        # Extract raw sequence from the current row up to the current column
        values_list = forecast_df.iloc[row, 1:forecast_df.columns.get_loc(column)].tolist()
        raw_seq = np.array(values_list).astype(float)
        # Check if the sum of values in raw_seq is zero
        if np.sum(raw_seq) == 0:
            yhat = 0.0
        else:
            # Extract the last n_steps values from the raw sequence
            x_input = raw_seq[-n_steps:]
            # Predict the next value using LSTM model
            yhat = lstm_univariate_prediction(raw_seq, n_steps, n_features, x_input)
            yhat = yhat[0][0].astype(float)
        # Update the corresponding cell in the DataFrame with the predicted value
        forecast_df.at[row, column] = yhat
        # Get the country name and year
        country_name = forecast_df.at[row, 'Country Name']
        year = column.split(' ')[0]
        # Print the country name, year, and predicted value
        print(f"{country_name} {year}: {yhat}")
        
print('Run Complete!')

Columns:   0%|          | 0/5 [00:00<?, ?it/s]

Rows:   0%|          | 0/266 [00:00<?, ?it/s]

/tmp/ipykernel_18/3265109729.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3471563756465912' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  forecast_df.at[row, column] = yhat
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Afghanistan 2026: 0.3471563756465912
Albania 2026: 2.5644686222076416
Algeria 2026: 6.920771598815918
American Samoa 2026: 0.0
Andorra 2026: 4.66353178024292
Angola 2026: 1.2791200876235962
Antigua and Barbuda 2026: 7.666146278381348
Argentina 2026: 7.475348949432373
Armenia 2026: 2.566344738006592
Aruba 2026: 0.0
Australia 2026: 15.140480041503906
Austria 2026: 4.589039325714111
Azerbaijan 2026: 2.2262797355651855
Bahamas, The 2026: 6.894532203674316
Bahrain 2026: 21.31487274169922
Bangladesh 2026: 0.3938363790512085
Barbados 2026: 5.720296382904053
Belarus 2026: 7.19505500793457
Belgium 2026: 6.120936393737793
Belize 2026: 1.3905246257781982
Benin 2026: 0.5227792859077454
Bermuda 2026: 0.0
Bhutan 2026: 14.99836254119873
Bolivia 2026: 4.159817695617676
Bosnia and Herzegovina 2026: 5.867332458496094
Botswana 2026: 2.7566568851470947
Brazil 2026: 6.605946063995361
British Virgin Islands 2026: 0.0
Brunei Darussalam 2026: 17.585399627685547
Bulgaria 2026: 4.29046630859375
Burkina Faso 202

Rows:   0%|          | 0/266 [00:00<?, ?it/s]

...

/tmp/ipykernel_18/3265109729.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.33656013011932373' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  forecast_df.at[row, column] = yhat
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Afghanistan 2029: 0.33656013011932373
Albania 2029: 3.0870635509490967
Algeria 2029: 8.834396362304688
American Samoa 2029: 0.0
Andorra 2029: 5.077916622161865
Angola 2029: 1.3669244050979614
Antigua and Barbuda 2029: 8.021599769592285
Argentina 2029: 6.765531063079834
Armenia 2029: 2.7970948219299316
Aruba 2029: 0.0
Australia 2029: 14.858656883239746
Austria 2029: 3.9444022178649902
Azerbaijan 2029: 2.06901216506958
Bahamas, The 2029: 6.87950325012207
Bahrain 2029: 21.470613479614258
Bangladesh 2029: 0.40404805541038513
Barbados 2029: 5.7686872482299805
Belarus 2029: 7.61550235748291
Belgium 2029: 5.112795352935791
Belize 2029: 1.3469232320785522
Benin 2029: 0.5293787717819214
Bermuda 2029: 0.0
Bhutan 2029: 72.22347259521484
Bolivia 2029: 5.947306156158447
Bosnia and Herzegovina 2029: 5.89607048034668
Botswana 2029: 2.9431211948394775
Brazil 2029: 10.864921569824219
British Virgin Islands 2029: 0.0
Brunei Darussalam 2029: 16.768579483032227
Bulgaria 2029: 3.760495662689209
Burkina Fas

Rows:   0%|          | 0/266 [00:00<?, ?it/s]

/tmp/ipykernel_18/3265109729.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3469204306602478' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  forecast_df.at[row, column] = yhat
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Afghanistan 2030: 0.3469204306602478
Albania 2030: 3.0087246894836426
Algeria 2030: 9.728252410888672
American Samoa 2030: 0.0
Andorra 2030: 4.169365406036377
Angola 2030: 1.4298076629638672
Antigua and Barbuda 2030: 8.624247550964355
Argentina 2030: 6.893901824951172
Armenia 2030: 2.94638991355896
Aruba 2030: 0.0
Australia 2030: 14.495553970336914
Austria 2030: 3.2644155025482178
Azerbaijan 2030: 2.0898513793945312
Bahamas, The 2030: 6.935649394989014
Bahrain 2030: 21.58684539794922
Bangladesh 2030: 0.40489599108695984
Barbados 2030: 5.682384967803955
Belarus 2030: 7.835473537445068
Belgium 2030: 4.9128241539001465
Belize 2030: 1.3426434993743896
Benin 2030: 0.5297925472259521
Bermuda 2030: 0.0
Bhutan 2030: 115.70030975341797
Bolivia 2030: 7.166015625
Bosnia and Herzegovina 2030: 5.888858795166016
Botswana 2030: 3.0703864097595215
Brazil 2030: 13.671385765075684
British Virgin Islands 2030: 0.0
Brunei Darussalam 2030: 17.219820022583008
Bulgaria 2030: 4.813060283660889
Burkina Faso 20

In [7]:
# Create submission file
submission_df = forecast_df[['Country Name', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]', '2030 [YR2030]']]
submission_df.to_csv('lstm_submission.csv', index=False)